In [1]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.feature_selection import RFECV

from sklearn.preprocessing import scale
from sklearn.decomposition import PCA

In [2]:
user = pd.read_csv("analysis_data.csv", sep=",", index_col="Client")
print(user.shape)
user.head()

(1581, 41)


,Age,Tenure,M,Count_CA,Count_SA,Count_MF,Count_OVD,Count_CC,Count_CL,ActBal_CA,...,Sale_CL,Revenue_MF,Revenue_CC,Revenue_CL,Actbal_CA_avg,ActBal_SA_avg,ActBal_MF_avg,ActBal_OVD_avg,ActBal_CC_avg,ActBal_CL_avg
Client,,,,,,,,,,,,,,,,,,,,,
909,21,27,1,1,0.0,0.0,1.0,0.0,1.0,4.710714,...,NaN,NaN,NaN,NaN,4.710714,0.00000,0.0,0.0,0.00,4291.996429
1217,38,165,1,1,0.0,0.0,0.0,0.0,0.0,6752.244643,...,0.0,0.000000,0.000000,0.0,6752.244643,0.00000,0.0,0.0,0.00,0.000000
850,49,44,0,1,0.0,0.0,0.0,0.0,0.0,43.523214,...,0.0,0.000000,0.893929,0.0,43.523214,0.00000,0.0,0.0,0.00,0.000000
1473,54,34,1,1,1.0,0.0,0.0,1.0,1.0,29.024286,...,0.0,4.929643,0.000000,0.0,29.024286,14447.80143,0.0,0.0,653.91,1132.590357
1038,29,106,1,1,0.0,0.0,0.0,0.0,0.0,27.035714,...,0.0,0.000000,0.000000,0.0,27.035714,0.00000,0.0,0.0,0.00,0.000000


In [3]:
user_tr = user[pd.notnull(user['Sale_MF'])]
print(user_tr.shape)
user_tr.head()

(947, 41)


,Age,Tenure,M,Count_CA,Count_SA,Count_MF,Count_OVD,Count_CC,Count_CL,ActBal_CA,...,Sale_CL,Revenue_MF,Revenue_CC,Revenue_CL,Actbal_CA_avg,ActBal_SA_avg,ActBal_MF_avg,ActBal_OVD_avg,ActBal_CC_avg,ActBal_CL_avg
Client,,,,,,,,,,,,,,,,,,,,,
1217,38,165,1,1,0.0,0.0,0.0,0.0,0.0,6752.244643,...,0.0,0.000000,0.000000,0.000000,6752.244643,0.00000,0.0,0.0000,0.00,0.000000
850,49,44,0,1,0.0,0.0,0.0,0.0,0.0,43.523214,...,0.0,0.000000,0.893929,0.000000,43.523214,0.00000,0.0,0.0000,0.00,0.000000
1473,54,34,1,1,1.0,0.0,0.0,1.0,1.0,29.024286,...,0.0,4.929643,0.000000,0.000000,29.024286,14447.80143,0.0,0.0000,653.91,1132.590357
1038,29,106,1,1,0.0,0.0,0.0,0.0,0.0,27.035714,...,0.0,0.000000,0.000000,0.000000,27.035714,0.00000,0.0,0.0000,0.00,0.000000
225,14,187,1,1,0.0,0.0,1.0,0.0,0.0,345.686071,...,1.0,0.000000,0.000000,8.130714,345.686071,0.00000,0.0,618.3975,0.00,0.000000


In [4]:
user_tr = user_tr.drop(['Sale_MF','Sale_CL','Revenue_MF','Revenue_CC','Revenue_CL'], axis=1)
user_tr.head()

,Age,Tenure,M,Count_CA,Count_SA,Count_MF,Count_OVD,Count_CC,Count_CL,ActBal_CA,...,TransactionsDebCash_Card,TransactionsDebCashless_Card,TransactionsDeb_PaymentOrder,Sale_CC,Actbal_CA_avg,ActBal_SA_avg,ActBal_MF_avg,ActBal_OVD_avg,ActBal_CC_avg,ActBal_CL_avg
Client,,,,,,,,,,,,,,,,,,,,,
1217,38,165,1,1,0.0,0.0,0.0,0.0,0.0,6752.244643,...,0,0,1,0.0,6752.244643,0.00000,0.0,0.0000,0.00,0.000000
850,49,44,0,1,0.0,0.0,0.0,0.0,0.0,43.523214,...,0,0,1,1.0,43.523214,0.00000,0.0,0.0000,0.00,0.000000
1473,54,34,1,1,1.0,0.0,0.0,1.0,1.0,29.024286,...,1,26,11,0.0,29.024286,14447.80143,0.0,0.0000,653.91,1132.590357
1038,29,106,1,1,0.0,0.0,0.0,0.0,0.0,27.035714,...,0,0,0,0.0,27.035714,0.00000,0.0,0.0000,0.00,0.000000
225,14,187,1,1,0.0,0.0,1.0,0.0,0.0,345.686071,...,0,4,1,0.0,345.686071,0.00000,0.0,618.3975,0.00,0.000000


In [5]:
print(user_tr.shape)

(947, 36)


In [6]:
user_tr_X = user_tr.drop('Sale_CC', axis=1)
user_tr_Y = user_tr['Sale_CC']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(user_tr_X, user_tr_Y, test_size=0.2, random_state=42)

In [8]:
logreg = LogisticRegression()
logreg.fit(X_train,y_train)


y_pred = logreg.predict(X_test)
print(roc_auc_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

y_pred_train = logreg.predict(X_train)
print(roc_auc_score(y_train, y_pred_train))
print(classification_report(y_train, y_pred_train))


0.589655172414
             precision    recall  f1-score   support

        0.0       0.80      0.98      0.88       145
        1.0       0.75      0.20      0.32        45

avg / total       0.79      0.79      0.75       190

0.573598820059
             precision    recall  f1-score   support

        0.0       0.78      0.98      0.87       565
        1.0       0.74      0.17      0.27       192

avg / total       0.77      0.77      0.72       757



In [9]:
logreg = LogisticRegression()
scores = cross_val_score(logreg, user_tr_X, user_tr_Y, cv=10, scoring='roc_auc', n_jobs=-1)
print("roc_auc: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

roc_auc: 0.62 (+/- 0.14)


In [10]:
svc = SVC(kernel='rbf', degree=2, random_state=101)
scores = cross_val_score(svc, user_tr_X, user_tr_Y, cv=10, scoring='roc_auc', n_jobs=-1)
print("roc_auc: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

roc_auc: 0.50 (+/- 0.02)


In [11]:
random_forest = RandomForestClassifier(n_estimators = 25)
scores = cross_val_score(random_forest, user_tr_X, user_tr_Y, cv=10, scoring='roc_auc', n_jobs=-1)
print("roc_auc: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

roc_auc: 0.61 (+/- 0.19)


In [12]:
gaussian = GaussianNB()
scores = cross_val_score(gaussian, user_tr_X, user_tr_Y, cv=10, scoring='roc_auc', n_jobs=-1)
print("roc_auc: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

roc_auc: 0.58 (+/- 0.10)


In [13]:
#Scale the 
user_tr_X_scale = scale(user_tr_X)
print(user_tr_X_scale)

[[-0.26748029  0.98414783  0.93952316 ..., -0.21893948 -0.20892838
  -0.21074605]
 [ 0.32422542 -0.85655318 -1.06436971 ..., -0.21893948 -0.20892838
  -0.21074605]
 [ 0.59318256 -1.00867723  0.93952316 ..., -0.21893948  3.97811872
   0.57597595]
 ..., 
 [-1.07435172  0.9080858   0.93952316 ..., -0.21893948 -0.20892838
  -0.21074605]
 [ 0.16285114 -0.6283671   0.93952316 ..., -0.21893948 -0.20892838
  -0.21074605]
 [ 0.96972256  1.10584707 -1.06436971 ..., -0.21893948 -0.20892838
  -0.21074605]]


In [14]:
logreg = LogisticRegression()
scores = cross_val_score(logreg, user_tr_X_scale, user_tr_Y, cv=10, scoring='roc_auc', n_jobs=-1)
print("roc_auc: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

roc_auc: 0.64 (+/- 0.16)


In [15]:
logreg = LogisticRegression(penalty='l2')
selector = RFECV(estimator=logreg, step=1, cv=10,scoring='roc_auc')

selector.fit(user_tr_X_scale, user_tr_Y)
print("Optimal number of features : %d" % selector.n_features_)


Optimal number of features : 34


In [16]:
user_tr_X_scale_transform = selector.transform(user_tr_X_scale)

In [17]:
search = {'penalty': ['l1','l2'],'C': [0.001, 0.1, 1, 10, 100, 1000], 'class_weight': ['balanced', None]}


logreg = LogisticRegression()

logreg_grid = GridSearchCV(estimator=logreg,param_grid=search, scoring='roc_auc', n_jobs=-1,refit=True, cv=10)
logreg_grid.fit(user_tr_X_scale_transform,user_tr_Y)
print(logreg_grid.best_estimator_)

print(logreg_grid.best_params_)

print(logreg_grid.best_score_)

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
{'C': 1, 'class_weight': None, 'penalty': 'l2'}
0.639011086244


In [18]:
svc = SVC(kernel='rbf', degree=2, random_state=101)
scores = cross_val_score(svc, user_tr_X_scale, user_tr_Y, cv=10, scoring='roc_auc', n_jobs=-1)
print("roc_auc: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
print(scores)

roc_auc: 0.60 (+/- 0.14)
[ 0.52934272  0.73826291  0.67957746  0.51232394  0.65492958  0.63732394
  0.59096244  0.60992039  0.50459277  0.58420086]


In [19]:
search = search_dict = {'kernel': ['linear','rbf'],'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001]}


svc = SVC()

svc_grid = RandomizedSearchCV(estimator=svc,param_distributions=search, scoring='roc_auc', n_jobs=-1,refit=True, cv=3)
svc_grid.fit(user_tr_X_scale,user_tr_Y)
print(svc_grid.best_estimator_)

print(svc_grid.best_params_)

print(svc_grid.best_score_)

SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.001, kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
{'kernel': 'linear', 'gamma': 0.001, 'C': 100}
0.620357379201


In [20]:
svc = SVC(kernel='linear', degree=2, random_state=101)
scores = cross_val_score(svc, user_tr_X_scale, user_tr_Y, cv=10, scoring='roc_auc', n_jobs=-1)
print(scores)
print("roc_auc: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

[ 0.54342723  0.73356808  0.70598592  0.57922535  0.68309859  0.65316901
  0.55575117  0.70545009  0.54562156  0.59216167]
roc_auc: 0.63 (+/- 0.14)


In [21]:
random_forest = RandomForestClassifier(n_estimators = 20)
scores = cross_val_score(random_forest, user_tr_X_scale, user_tr_Y, cv=10, scoring='roc_auc', n_jobs=-1)
print("roc_auc: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

roc_auc: 0.60 (+/- 0.15)


In [22]:
gaussian = GaussianNB()
scores = cross_val_score(gaussian, user_tr_X_scale, user_tr_Y, cv=10, scoring='roc_auc', n_jobs=-1)
print("roc_auc: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

roc_auc: 0.60 (+/- 0.12)


In [23]:
#dimension reduction
pca = PCA(n_components=20)
X_pca_20 = pca.fit_transform(user_tr_X_scale)
X_pca_20.shape

(947, 20)

In [24]:
svc = SVC(kernel='linear', degree=2, random_state=101)
scores = cross_val_score(svc, X_pca_20, user_tr_Y, cv=10, scoring='roc_auc', n_jobs=-1)
print(scores)
print("roc_auc: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

[ 0.54812207  0.72652582  0.68192488  0.56866197  0.6971831   0.61326291
  0.53286385  0.63380282  0.55725658  0.50214329]
roc_auc: 0.61 (+/- 0.15)


In [25]:
#with penalty
logreg = LogisticRegression(penalty='l1', C=1)
scores = cross_val_score(logreg, X_pca_20, user_tr_Y, cv=20, scoring='roc_auc', n_jobs=-1)
print(scores)
print("roc_auc: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

[ 0.62962963  0.57175926  0.7662037   0.77546296  0.74074074  0.81481481
  0.60185185  0.50694444  0.73611111  0.60648148  0.65952381  0.65952381
  0.57142857  0.56904762  0.67619048  0.61428571  0.4047619   0.74025974
  0.44675325  0.50909091]
roc_auc: 0.63 (+/- 0.22)


In [26]:
random_forest = RandomForestClassifier(n_estimators = 50)
scores = cross_val_score(random_forest, X_pca_20, user_tr_Y, cv=10, scoring='roc_auc', n_jobs=-1)
print("roc_auc: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

roc_auc: 0.62 (+/- 0.14)


In [27]:
svc_lin = LinearSVC()
scores = cross_val_score(svc_lin, X_pca_20, user_tr_Y, cv=10, scoring='roc_auc', n_jobs=-1)
print(scores)
print("roc_auc: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

[ 0.57981221  0.79107981  0.75821596  0.54870892  0.67077465  0.65140845
  0.55692488  0.67299449  0.53704838  0.50765462]
roc_auc: 0.63 (+/- 0.18)


In [28]:
gaussian = GaussianNB()
scores = cross_val_score(gaussian, X_pca_20, user_tr_Y, cv=10, scoring='roc_auc', n_jobs=-1)
print("roc_auc: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

roc_auc: 0.58 (+/- 0.14)


In [29]:
search = {'penalty': ['l1','l2'],'C': [0.001, 0.1, 1, 10, 100, 1000], 'class_weight': ['balanced', None]}


logreg = LogisticRegression()

logreg_grid = GridSearchCV(estimator=logreg,param_grid=search, scoring='roc_auc', n_jobs=-1,refit=True, cv=10)
logreg_grid.fit(X_pca_20,user_tr_Y)
print(logreg_grid.best_estimator_)

print(logreg_grid.best_params_)

print(logreg_grid.best_score_)

LogisticRegression(C=1, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l1', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)
{'C': 1, 'class_weight': 'balanced', 'penalty': 'l1'}
0.62789075869
